<h1 style="color:#1398A1">TRAINING<h1>


<h3 style="color:#CCC229">LIBRAIRIES IMPORT<h3>


In [3]:
import sys
from pathlib import Path

src_path = Path("..")
sys.path.append(str(src_path))
%load_ext autoreload
%autoreload 2

from app.src.agent.utils.infos import list_snake_models
from app.src.agent.training.train import train_snake

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h3 style="color:#CCC229">SOME TESTS<h3>

In [4]:
# Example of listing all available Snake models
list_snake_models()

🔍 Recherche des modèles dans snakeRL/snake-rl-models...

             MODELE (UUID)                   ||                 DESCRIPTION
--------------------------------------------------------------------------------------------------------------
08x08 / 69b21c9f-56e7-433d-b9eb-fa60abc47c87 || Algo: PPO | Date: 25/01/2026 23:38:26 | Reward: 4.71
07x07 / ae8f8253-51c7-4744-950c-18715e4c0e63 || Algo: PPO | Date: 25/01/2026 22:44:48 | Reward: -0.12
07x07 / 12cf02a3-826b-437c-8ed0-18496e43b58a || Algo: PPO | Date: 25/01/2026 22:44:12 | Reward: -0.99
07x07 / a752e28b-b125-4a7a-a042-3fa15478e0b9 || Algo: PPO | Date: 25/01/2026 22:42:13 | Reward: -0.77
07x07 / 2fec9971-ebfe-42cf-8f4d-1e9e88d2e398 || Algo: PPO | Date: 25/01/2026 22:41:34 | Reward: -0.93
07x07 / 0761432b-54d2-49c2-8b4f-91e6021c07ff || Algo: PPO | Date: 25/01/2026 22:33:29 | Reward: -0.70
07x07 / 76344a9d-fce8-4661-b4fe-026da16826ea || Algo: PPO | Date: 25/01/2026 22:30:37 | Reward: -0.78
07x07 / 98970536-7328-4493-a0f5-b9d4d2c6001

In [5]:
# Example of training a Snake agent using PPO algorithm
train_snake(timesteps=1_000_000, grid_size=5, n_envs=10, algorithm="PPO")

✨ Création d'un nouvel agent vierge sur grille 5x5.


2026/01/26 11:12:00 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/01/26 11:12:00 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/01/26 11:12:00 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/01/26 11:12:00 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/01/26 11:12:00 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/01/26 11:12:00 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/01/26 11:12:02 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/26 11:12:03 INFO mlflow.store.db.utils: Updating database tables
2026/01/26 11:12:03 INFO alembic.runtime.migration: Context impl PostgresqlImpl.
2026/01/26 11:12:03 INFO alembic.runtime.migration: Will assume transactional DDL.
2026/01/26 11:12:04 INFO alembic.runtime.migration: Context impl PostgresqlImpl.
2026/01/26 11:12:04 INFO alembic.run


Démarrage Run MLflow : NEW_TRAINING_20260126_111159_ddfab7de
Nouvel ID : ddfab7de-d0f0-4e44-a59b-0ac0604aba0d
Using cpu device

Démarrage de l'entraînement pour 1000000 timesteps...
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 9.09     |
|    ep_rew_mean     | -0.808   |
| time/              |          |
|    fps             | 11537    |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 20480    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.18        |
|    ep_rew_mean          | -0.718      |
| time/                   |             |
|    fps                  | 2626        |
|    iterations           | 2           |
|    time_elapsed         | 15          |
|    total_timesteps      | 40960       |
| train/                  |             |
|    approx_kl            | 0.010932246 |
|    clip_fract

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

2026/01/26 11:25:23 WARNING mlflow.pyfunc: Failed to infer model signature: Type hint <input: <class 'numpy.ndarray'>, output: None> cannot be used to infer model signature and input example is not provided, model signature cannot be inferred.
2026/01/26 11:25:26 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/26 11:25:26 INFO mlflow.store.db.utils: Updating database tables
2026/01/26 11:25:27 INFO alembic.runtime.migration: Context impl PostgresqlImpl.
2026/01/26 11:25:27 INFO alembic.runtime.migration: Will assume transactional DDL.
Registered model 'Snake_5x5' already exists. Creating a new version of this model...
Created version '5' of model 'Snake_5x5'.


Terminé ! Nouveau modèle disponible : ddfab7de-d0f0-4e44-a59b-0ac0604aba0d


In [ ]:
# Example of resuming training from a previous model using its UUID
train_snake(timesteps=2_000_000, n_envs=10, algorithm="PPO", base_uuid="ddfab7de-d0f0-4e44-a59b-0ac0604aba0d")

Tentative de récupération du modèle ddfab7de-d0f0-4e44-a59b-0ac0604aba0d...
Scan du dépôt pour trouver l'UUID : ddfab7de-d0f0-4e44-a59b-0ac0604aba0d ...
Fichier trouvé : 5x5/ddfab7de-d0f0-4e44-a59b-0ac0604aba0d/metadata.json


metadata.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

Téléchargement du modèle : 5x5/ddfab7de-d0f0-4e44-a59b-0ac0604aba0d/model.zip


5x5/ddfab7de-d0f0-4e44-a59b-0ac0604aba0d(…):   0%|          | 0.00/175k [00:00<?, ?B/s]

✅ Succès ! Agent chargé (Grille 5x5)
✅ Modèle chargé avec succès. Reprise sur grille 5x5.

Démarrage Run MLflow : FINE-TUNING_20260126_113131_5cc88d67
Nouvel ID : 5cc88d67-1e48-40ae-943b-6528780da426

Démarrage de l'entraînement pour 2000000 timesteps...
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 34.1     |
|    ep_rew_mean     | 4.41     |
| time/              |          |
|    fps             | 7538     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 1024000  |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 32.1       |
|    ep_rew_mean          | 4.04       |
| time/                   |            |
|    fps                  | 2088       |
|    iterations           | 2          |
|    time_elapsed         | 19         |
|    total_timesteps      | 1044480    |
| train/                  |         